In [1]:
import pandas as pd
import torch.nn as nn
import pickle
import torch
from torchvision import models
from torchvision.models import detection, resnet50, ResNet50_Weights
import os
import numpy as np
import cv2
from torchvision import transforms
from sklearn.preprocessing import LabelEncoder
import scipy

In [2]:
CONFIGS = {
    # determine the current device and based on that set the pin memory
    # flag
    "DEVICE": "cuda" if torch.cuda.is_available() else "cpu",
    # specify ImageNet mean and standard deviation
    "IMG_MEAN": [0.485, 0.456, 0.406],
    "IMG_STD": [0.229, 0.224, 0.225],
    "MC_DROPOUT_ENABLED": False,  # Switch to enable/disable MC Dropout for confidence score
    "NUM_DROPOUT_RUNS": 3,
    "CONFIDENCE_THRESHOLD": 0,
    "BIG_MODEL_IMG_SIZE": 320,
    "SMALL_MODEL_IMG_SIZE": 60,
    "MEAN_PRIOR": -15,
    "MODEL_PATH": 'traindatawithin1'
}

# Big model

## Model loading

In [3]:
class MultiHeadResNet_BigModel(nn.Module):
    def __init__(self, num_classes_prdtype, num_classes_weight, num_classes_halal, num_classes_healthy):
        super(MultiHeadResNet_BigModel, self).__init__()
        self.base_model = models.resnet50(weights=ResNet50_Weights.DEFAULT)
        num_ftrs = self.base_model.fc.in_features
        self.base_model.fc = nn.Identity()

        # Define custom fully connected layers for each prediction head
        self.fc_prdtype = nn.Linear(num_ftrs, num_classes_prdtype)
        self.fc_weight = nn.Linear(num_ftrs, num_classes_weight)
        self.fc_halal = nn.Linear(num_ftrs, num_classes_halal)
        self.fc_healthy = nn.Linear(num_ftrs, num_classes_healthy)
        self.fc_bbox = nn.Linear(num_ftrs, 4)

    def forward(self, x):
        x = self.base_model(x)
        prdtype = self.fc_prdtype(x)
        weight = self.fc_weight(x)
        halal = self.fc_halal(x)
        healthy = self.fc_healthy(x)
        box = self.fc_bbox(x)
        return prdtype, weight, halal, healthy, box

    
# load label encoder 
def load_label_encoder_big_model():
    le_prdtype = pickle.loads(open(os.path.join('../../NN_model/model_weights', CONFIGS['MODEL_PATH'], 'le_prdtype.pickle'), "rb").read())
    le_weight = pickle.loads(open(os.path.join('../../NN_model/model_weights', CONFIGS['MODEL_PATH'], 'le_weight.pickle'), "rb").read())
    le_halal = pickle.loads(open(os.path.join('../../NN_model/model_weights', CONFIGS['MODEL_PATH'], 'le_halal.pickle'), "rb").read())
    le_healthy = pickle.loads(open(os.path.join('../../NN_model/model_weights', CONFIGS['MODEL_PATH'], 'le_healthy.pickle'), "rb").read())
    
    return le_prdtype, le_weight, le_halal, le_healthy

le_prdtype, le_weight, le_halal, le_healthy = load_label_encoder_big_model()

# Load the trained MultiHeadResNet model
def load_model():
    # Verify the number of classes for each label
    num_classes_prdtype = len(le_prdtype.classes_)
    num_classes_weight = len(le_weight.classes_)
    num_classes_halal = len(le_halal.classes_)
    num_classes_healthy = len(le_healthy.classes_)
    # print(num_classes_prdtype)
    # print(num_classes_healthy)

    custom_resnet_model = MultiHeadResNet_BigModel(
        num_classes_prdtype=num_classes_prdtype,
        num_classes_weight=num_classes_weight,
        num_classes_halal=num_classes_halal,
        num_classes_healthy=num_classes_healthy
    )

    model_path = os.path.join('../../NN_model/model_weights', CONFIGS['MODEL_PATH'], 'multi_head_model.pth')
    # print("test1")
    if os.path.exists(model_path):
        custom_resnet_model.load_state_dict(torch.load(model_path, map_location=CONFIGS['DEVICE']))
    else:
        raise FileNotFoundError(f"Model file not found: {model_path}")
    # print("test2")
    custom_resnet_model.to(CONFIGS['DEVICE'])
    custom_resnet_model.eval()
    return custom_resnet_model

big_model = load_model()

/Users/liupeng/anaconda3/envs/py38/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


## Scoring on main imgs

In [4]:
main_imgs_results_big_model = pd.read_csv(os.path.join('../../NN_model/model_weights', CONFIGS['MODEL_PATH'], 'main_imgs_results_big_model.csv'))
main_imgs_results_big_model.head()

,Filename,CorrectTotalLabel,ProductType_BabyMilkPowder,ProductType_Babyfood,ProductType_BeehoonVermicelli,ProductType_BiscuitsCrackersCookies,ProductType_BreakfastCereals,ProductType_CannedBakedBeans,ProductType_CannedBeefOtherMeats,ProductType_CannedBraisedPeanuts,...,Weight_400-499g,Weight_500-599g,Weight_600-699g,Weight_700-799g,Weight_800-899g,Weight_900-999g,HalalStatus_Halal,HalalStatus_NonHalal,HealthStatus_Healthy,HealthStatus_NonHealthy
0,20240123_2_0504.jpg,CannedSardingMackerel_100-199g_NonHalal_NonHea...,-1.882743,-2.266367,-1.672664,-2.738889,-2.199730,-1.376669,-2.124947,-2.212128,...,-1.235242,-0.939662,-0.806197,-0.798956,-0.888551,-0.792601,-1.465810,0.279448,-1.207268,0.371232
1,WIN_20231207_11_27_37_Pro.jpg,RiceBrownOthers_700-799g_NonHalal_Healthy,-1.523127,-1.651428,-2.044988,-1.407652,-2.493966,-1.889255,-1.845285,-2.128245,...,-0.923595,-0.847409,-0.678081,5.562907,-0.763209,-0.617778,-1.457077,0.446514,0.298923,-1.470801
2,20240123_5_1128.jpg,OtherBakingNeeds_100-199g_Halal_NonHealthy,-2.006237,-1.611458,-1.631186,-2.330615,-1.551032,-2.079339,-2.072249,-1.731458,...,-0.690004,-0.526840,-0.806302,-0.842093,-0.919830,-0.783886,0.541475,-0.961581,-1.156031,0.460518
3,20240123_2_0592.jpg,Sugar_200-299g_Halal_NonHealthy,-2.838345,-1.860340,-1.959939,-2.634158,-2.224911,-2.149055,-1.958104,-2.155508,...,-0.627989,-1.568842,-0.804087,-1.015083,-0.685697,-1.011046,0.518390,-1.466795,-1.420788,0.451316
4,IMG_5672-1-_jpeg.rf.378eb6bd1cf2d5647e1faf8105...,InstantNoodlesMultipack_400-499g_Halal_NonHealthy,-1.748531,-1.822276,-1.510194,-2.287951,-1.505609,-1.760424,-1.651896,-1.577105,...,4.283346,-0.740748,-0.762922,-0.857974,-0.828528,-0.787468,0.751286,-0.887367,-1.101639,0.543283


In [5]:
# Create a copy of the current column names to a list
new_columns = main_imgs_results_big_model.columns.tolist()

# Modify the first two elements
new_columns[0] = 'filepath'
new_columns[1] = 'label'

# Assign the modified list of column names back to the DataFrame
main_imgs_results_big_model.columns = new_columns


In [6]:
main_imgs_results_big_model.shape

(3598, 86)

# Scoring on unused training imgs

In [7]:
master_df = pd.read_csv("../../master_list.csv")
master_df = master_df[master_df['remove']!=1]
master_df.reset_index(drop=True, inplace=True)
master_df.head()

,filepath,xmin,ymin,xmax,ymax,label,ProductType,Weight,HalalStatus,HealthStatus,new_camera,tag,remove
0,IMG_20230428_123528_jpg.rf.5687b7b914f6d9aa98c...,151,42,497,591,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN,NaN
1,IMG_20230428_123522_jpg.rf.204ff37f497f2dce442...,88,81,442,567,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN,NaN
2,IMG_20230428_123708_jpg.rf.141ecd0cefaea75c0b7...,35,34,492,622,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN,NaN
3,IMG_20230428_123521_jpg.rf.1069b402272252862ec...,99,122,428,587,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN,NaN
4,IMG_20230428_123659_jpg.rf.5e1b6c4caabe48cf360...,103,17,474,592,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN,NaN


In [8]:
a = set(main_imgs_results_big_model["filepath"].tolist())
b = set(master_df["filepath"].tolist())
pending_imgs = [i for i in b if i not in a]
len(pending_imgs)

3130

In [9]:
transforms_test = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean=CONFIGS['IMG_MEAN'], std=CONFIGS['IMG_STD'])
])

new_imgs_results_big_model = []  # List to store the results

for row in pending_imgs:
    image_path = "../../rshiny/www/all_images/" + row
    frame = cv2.imread(image_path)

    # Preprocessing steps
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (CONFIGS['BIG_MODEL_IMG_SIZE'], CONFIGS['BIG_MODEL_IMG_SIZE']))
    frame = frame.transpose((2, 0, 1))
    frame = torch.from_numpy(frame).float()
    frame = transforms_test(frame).unsqueeze(0).to(CONFIGS['DEVICE'])

    # Perform prediction
    with torch.no_grad():
        out1, out2, out3, out4, _ = big_model(frame)

    # reference the correct label from master list
    tmp_correct_label = master_df.loc[master_df['filepath'] == row, 'label'].iloc[0]
        
    # Extract and store the results
    prediction_row = [row, tmp_correct_label]
    prediction_row.extend(out1.cpu().numpy().flatten())
    prediction_row.extend(out2.cpu().numpy().flatten())
    prediction_row.extend(out3.cpu().numpy().flatten())
    prediction_row.extend(out4.cpu().numpy().flatten())
    new_imgs_results_big_model.append(prediction_row)


# Define column names for the new DataFrame
column_names = ['filepath', 'label']
big_model_pred_col_name_original = main_imgs_results_big_model.columns[2:].tolist()
column_names += big_model_pred_col_name_original

# Create the DataFrame
new_imgs_results_big_model = pd.DataFrame(new_imgs_results_big_model, columns=column_names)
new_imgs_results_big_model.head()

,filepath,label,ProductType_BabyMilkPowder,ProductType_Babyfood,ProductType_BeehoonVermicelli,ProductType_BiscuitsCrackersCookies,ProductType_BreakfastCereals,ProductType_CannedBakedBeans,ProductType_CannedBeefOtherMeats,ProductType_CannedBraisedPeanuts,...,Weight_400-499g,Weight_500-599g,Weight_600-699g,Weight_700-799g,Weight_800-899g,Weight_900-999g,HalalStatus_Halal,HalalStatus_NonHalal,HealthStatus_Healthy,HealthStatus_NonHealthy
0,20240219_14_0466.jpg,PastaSauce_600-699g_Halal_NonHealthy,-1.974275,-1.202922,-2.283831,-2.873749,-2.228754,-2.620345,-2.273585,-2.127606,...,0.467087,0.327965,-0.914167,-0.594484,-1.238367,-0.888502,0.156181,-0.893878,-0.904001,-0.406066
1,20240301_2_0167.jpg,DriedFruits_100-199g_NonHalal_NonHealthy,-1.638000,-1.361197,-0.984796,-3.294581,-1.985519,-2.236452,-1.829604,-1.617829,...,-0.599405,-0.567084,-0.795779,-0.984034,-0.598614,-0.545151,-0.216936,-0.370638,-1.324195,0.248537
2,20240209_6_0438.jpg,BeehoonVermicelli_300-399g_NonHalal_NonHealthy,-1.745491,-2.202712,-1.099025,0.377193,0.577641,-1.521806,-1.564005,-2.046075,...,1.057924,-0.034718,-0.805295,-0.830255,0.105779,-0.324167,0.227881,-0.850632,-1.532716,0.458284
3,20240209_8_0058.jpg,BeehoonVermicelli_0-99g_NonHalal_NonHealthy,-1.960900,-1.518455,1.780049,-1.156224,-1.294438,-1.649794,-1.703447,-1.242785,...,-1.347317,-0.091430,-0.571852,-0.856631,-0.714548,-0.653533,-0.486588,0.266079,-1.463478,0.266805
4,20240219_9_1405.jpg,OtherSauceDressing_0-99g_Halal_NonHealthy,-1.624062,-2.318985,-1.679193,-2.192590,-1.693435,-0.959268,-2.677913,-2.417692,...,-1.038309,-1.685495,-0.907289,-0.031082,-1.249512,-0.403145,-0.630365,-0.265577,-1.521041,0.247634


## Scoring on new imgs

In [10]:
# new_imgs_df = pd.read_csv("GPT_model/chatgpt_prediction.csv")
# new_imgs_df.head()

In [11]:
# transforms_test = transforms.Compose([
#     transforms.ToPILImage(),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=CONFIGS['IMG_MEAN'], std=CONFIGS['IMG_STD'])
# ])

In [12]:
# master_df = pd.read_csv("../../master_list.csv")
# master_df.head()

In [13]:
# new_imgs_results_big_model = []  # List to store the results

# for idx, row in new_imgs_df.iterrows():
#     image_path = "../../all_images/" + row['img_filename']
#     frame = cv2.imread(image_path)

#     # Preprocessing steps
#     frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     frame = cv2.resize(frame, (CONFIGS['BIG_MODEL_IMG_SIZE'], CONFIGS['BIG_MODEL_IMG_SIZE']))
#     frame = frame.transpose((2, 0, 1))
#     frame = torch.from_numpy(frame).float()
#     frame = transforms_test(frame).unsqueeze(0).to(CONFIGS['DEVICE'])

#     # Perform prediction
#     with torch.no_grad():
#         out1, out2, out3, out4, _ = big_model(frame)

#     # reference the correct label from master list
#     tmp_correct_label = master_df.loc[master_df['filepath'] == row['img_filename'], 'label'].iloc[0]
        
#     # Extract and store the results
#     prediction_row = [row['img_filename'], tmp_correct_label]
#     prediction_row.extend(out1.cpu().numpy().flatten())
#     prediction_row.extend(out2.cpu().numpy().flatten())
#     prediction_row.extend(out3.cpu().numpy().flatten())
#     prediction_row.extend(out4.cpu().numpy().flatten())
#     new_imgs_results_big_model.append(prediction_row)


# # Define column names for the new DataFrame
# column_names = ['filepath', 'label']
# big_model_pred_col_name_original = main_imgs_results_big_model.columns[2:].tolist()
# column_names += big_model_pred_col_name_original

# # Create the DataFrame
# new_imgs_results_big_model = pd.DataFrame(new_imgs_results_big_model, columns=column_names)
# new_imgs_results_big_model.head()

In [14]:
# new_imgs_results_big_model.shape

## All scorings from big model

In [15]:
main_imgs_results_big_model['img_type'] = "existing"
new_imgs_results_big_model['img_type'] = "new"
all_imgs_results_big_model = pd.concat([main_imgs_results_big_model, new_imgs_results_big_model], axis=0)
all_imgs_results_big_model.reset_index(drop=True, inplace=True)
all_imgs_results_big_model.head()
# all_imgs_results_big_model = main_imgs_results_big_model

,filepath,label,ProductType_BabyMilkPowder,ProductType_Babyfood,ProductType_BeehoonVermicelli,ProductType_BiscuitsCrackersCookies,ProductType_BreakfastCereals,ProductType_CannedBakedBeans,ProductType_CannedBeefOtherMeats,ProductType_CannedBraisedPeanuts,...,Weight_500-599g,Weight_600-699g,Weight_700-799g,Weight_800-899g,Weight_900-999g,HalalStatus_Halal,HalalStatus_NonHalal,HealthStatus_Healthy,HealthStatus_NonHealthy,img_type
0,20240123_2_0504.jpg,CannedSardingMackerel_100-199g_NonHalal_NonHea...,-1.882743,-2.266367,-1.672664,-2.738889,-2.199730,-1.376669,-2.124947,-2.212128,...,-0.939662,-0.806197,-0.798956,-0.888551,-0.792601,-1.465810,0.279448,-1.207268,0.371232,existing
1,WIN_20231207_11_27_37_Pro.jpg,RiceBrownOthers_700-799g_NonHalal_Healthy,-1.523127,-1.651428,-2.044988,-1.407652,-2.493966,-1.889255,-1.845285,-2.128245,...,-0.847409,-0.678081,5.562907,-0.763209,-0.617778,-1.457077,0.446514,0.298923,-1.470801,existing
2,20240123_5_1128.jpg,OtherBakingNeeds_100-199g_Halal_NonHealthy,-2.006237,-1.611458,-1.631186,-2.330615,-1.551032,-2.079339,-2.072249,-1.731458,...,-0.526840,-0.806302,-0.842093,-0.919830,-0.783886,0.541475,-0.961581,-1.156031,0.460518,existing
3,20240123_2_0592.jpg,Sugar_200-299g_Halal_NonHealthy,-2.838345,-1.860340,-1.959939,-2.634158,-2.224911,-2.149055,-1.958104,-2.155508,...,-1.568842,-0.804087,-1.015083,-0.685697,-1.011046,0.518390,-1.466795,-1.420788,0.451316,existing
4,IMG_5672-1-_jpeg.rf.378eb6bd1cf2d5647e1faf8105...,InstantNoodlesMultipack_400-499g_Halal_NonHealthy,-1.748531,-1.822276,-1.510194,-2.287951,-1.505609,-1.760424,-1.651896,-1.577105,...,-0.740748,-0.762922,-0.857974,-0.828528,-0.787468,0.751286,-0.887367,-1.101639,0.543283,existing


In [16]:
new_imgs_results_big_model.shape

(3130, 87)

In [17]:
main_imgs_results_big_model.shape

(3598, 87)

In [18]:
all_imgs_results_big_model.tail()

,filepath,label,ProductType_BabyMilkPowder,ProductType_Babyfood,ProductType_BeehoonVermicelli,ProductType_BiscuitsCrackersCookies,ProductType_BreakfastCereals,ProductType_CannedBakedBeans,ProductType_CannedBeefOtherMeats,ProductType_CannedBraisedPeanuts,...,Weight_500-599g,Weight_600-699g,Weight_700-799g,Weight_800-899g,Weight_900-999g,HalalStatus_Halal,HalalStatus_NonHalal,HealthStatus_Healthy,HealthStatus_NonHealthy,img_type
6723,20240219_5_0591.jpg,OtherSauceDressing_500-599g_NonHalal_NonHealthy,-2.242272,-1.716983,-2.830695,-2.382903,-1.560732,-2.380832,-2.606949,-2.341588,...,-0.431033,-0.795907,-0.658776,-0.518073,-1.272201,-0.544591,-0.040973,-1.433059,0.340987,new
6724,20240209_1_0906.jpg,Potatochips_100-199g_NonHalal_NonHealthy,-1.995471,-0.532489,-1.351002,0.276491,-2.000114,-1.709023,-1.926744,-2.055859,...,0.401323,-0.367979,-0.683025,-0.950073,-0.330110,0.091763,-0.384905,-1.046988,-0.024196,new
6725,20240209_3_0137.jpg,SweetsChocolatesOthers_0-99g_NonHalal_Nonhealthy,-1.784720,-1.807640,0.166395,0.015898,-1.375056,-1.599759,-1.524860,-1.694406,...,-1.072458,-1.499573,-0.821048,-0.568135,-0.144502,-0.891586,0.189837,-1.345261,0.364146,new
6726,20240301_5_0163.jpg,Potatochips_0-99g_Halal_NonHealthy,-2.088190,-1.883071,-1.274734,0.162747,-2.138230,-1.750648,-2.089661,-1.881107,...,-1.003887,-0.911979,-0.981586,-0.971958,-1.040804,0.039545,-0.502021,-0.744780,-0.342415,new
6727,20240301_1_0604.jpg,SweetsChocolatesOthers_100-199g_NonHalal_NonHe...,-2.260824,-1.245013,-1.659275,-1.844569,-2.504773,-2.338939,-2.198458,-2.294300,...,-0.967540,1.155558,-0.093337,-1.151238,-0.658425,0.033894,-0.269218,-1.089769,-0.319919,new


In [19]:
all_imgs_results_big_model.shape

(6728, 87)

In [20]:
all_imgs_results_big_model.to_csv(os.path.join('../../NN_model/model_weights', CONFIGS['MODEL_PATH'], 'all_imgs_results_big_model.csv'), index=True)

In [21]:
# class MultiHeadResNet_SmallModel(nn.Module):
#     def __init__(self, num_classes_prdtype, num_classes_weight, num_classes_halal, num_classes_healthy):
#         super(MultiHeadResNet_SmallModel, self).__init__()
#         self.base_model = models.resnet18(pretrained=True)
#         num_ftrs = self.base_model.fc.in_features
#         self.base_model.fc = nn.Identity()

#         # Define custom fully connected layers for each prediction head
#         self.fc_prdtype = nn.Linear(num_ftrs, num_classes_prdtype)
#         self.fc_weight = nn.Linear(num_ftrs, num_classes_weight)
#         self.fc_halal = nn.Linear(num_ftrs, num_classes_halal)
#         self.fc_healthy = nn.Linear(num_ftrs, num_classes_healthy)

#     def forward(self, x):
#         x = self.base_model(x)
#         prdtype = self.fc_prdtype(x)
#         weight = self.fc_weight(x)
#         halal = self.fc_halal(x)
#         healthy = self.fc_healthy(x)
#         return prdtype, weight, halal, healthy

    
# # load label encoder 
# def load_label_encoder_small_model():
#     le_prdtype = pickle.loads(open("../small_model/output/le_prdtype.pickle", "rb").read())
#     le_weight = pickle.loads(open("../small_model/output/le_weight.pickle", "rb").read())
#     le_halal = pickle.loads(open("../small_model/output/le_halal.pickle", "rb").read())
#     le_healthy = pickle.loads(open("../small_model/output/le_healthy.pickle", "rb").read())
    
#     return le_prdtype, le_weight, le_halal, le_healthy

# le_prdtype, le_weight, le_halal, le_healthy = load_label_encoder_small_model()

# # Load the trained MultiHeadResNet model
# def load_model():
#     # Verify the number of classes for each label
#     num_classes_prdtype = len(le_prdtype.classes_)
#     num_classes_weight = len(le_weight.classes_)
#     num_classes_halal = len(le_halal.classes_)
#     num_classes_healthy = len(le_healthy.classes_)
#     # print(num_classes_prdtype)
#     # print(num_classes_healthy)

#     custom_resnet_model = MultiHeadResNet_SmallModel(
#         num_classes_prdtype=num_classes_prdtype,
#         num_classes_weight=num_classes_weight,
#         num_classes_halal=num_classes_halal,
#         num_classes_healthy=num_classes_healthy
#     )

#     model_path = '../small_model/output/multi_head_model.pth'
#     # print("test1")
#     if os.path.exists(model_path):
#         custom_resnet_model.load_state_dict(torch.load(model_path, map_location=CONFIGS['DEVICE']))
#     else:
#         raise FileNotFoundError(f"Model file not found: {model_path}")
#     # print("test2")
#     custom_resnet_model.to(CONFIGS['DEVICE'])
#     custom_resnet_model.eval()
#     return custom_resnet_model
 
# small_model = load_model()

In [22]:
# new_imgs_df = pd.read_csv("../small_model/new_imgs_list.csv")
# new_imgs_df.reset_index(drop=True, inplace=True)

# # ADHOC: change the new imgs to existing type
# new_imgs_df['label'] = 'AdultMilk_1-99g_Halal_NonHealthy'
# new_imgs_df['ProductType'] = 'AdultMilk'
# new_imgs_df['Weight'] = '1-99g'
# new_imgs_df['HalalStatus'] = 'Halal'
# new_imgs_df['HealthStatus'] = 'NonHealthy'

# new_imgs_df.head()

In [23]:
# new_imgs_results_small_model = pd.read_csv("../small_model/new_imgs_results_small_model.csv")
# new_imgs_results_small_model = new_imgs_results_small_model.loc[new_imgs_results_small_model.Filename.isin(new_imgs_df.filepath)]
# new_imgs_results_small_model.reset_index(drop=True, inplace=True)
# new_imgs_results_small_model.head()

In [24]:
# # Create a copy of the current column names to a list
# new_columns = new_imgs_results_small_model.columns.tolist()

# # Modify the first two elements
# new_columns[0] = 'filepath'
# new_columns[1] = 'label'

# # Assign the modified list of column names back to the DataFrame
# new_imgs_results_small_model.columns = new_columns
# new_imgs_results_small_model.head()

In [25]:
# # Check if any name from 'extracted_names' is not in 'df' and add it as a new column
# new_prdtype = list(set(all_imgs_results_big_model.columns) - set(new_imgs_results_small_model.columns))

# if len(new_prdtype)>0:
#     for col in new_prdtype:
#         new_imgs_results_small_model[col] = np.random.normal(loc=CONFIGS["MEAN_PRIOR"], scale=np.sqrt(0.1), size=new_imgs_results_small_model.shape[0])  # Initialize new columns

# new_imgs_results_small_model.head()  # Display the updated DataFrame for verificatio

In [26]:
# main_imgs_master_list = pd.read_csv("../master_list.csv")
# main_imgs_master_list.head()

In [27]:
# main_imgs_results_small_model = []  # List to store the results
# le_prdtype, le_weight, le_halal, le_healthy = load_label_encoder_small_model()

# for idx, row in main_imgs_master_list.iterrows():
#     image_path = "../all_images/" + row['filepath']
#     frame = cv2.imread(image_path)

#     # Preprocessing steps
#     frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     frame = cv2.resize(frame, (CONFIGS['SMALL_MODEL_IMG_SIZE'], CONFIGS['SMALL_MODEL_IMG_SIZE']))
#     frame = frame.transpose((2, 0, 1))
#     frame = torch.from_numpy(frame).float()
#     frame = transforms_test(frame).unsqueeze(0).to(CONFIGS['DEVICE'])

#     # Perform prediction
#     with torch.no_grad():
#         out1, out2, out3, out4 = small_model(frame)
    
#     # Extract and store the results
#     prediction_row = [row['filepath'], row['label']]
#     prediction_row.extend(out1.cpu().numpy().flatten())
#     prediction_row.extend(out2.cpu().numpy().flatten())
#     prediction_row.extend(out3.cpu().numpy().flatten())
#     prediction_row.extend(out4.cpu().numpy().flatten())
#     main_imgs_results_small_model.append(prediction_row)


# # Define column names for the new DataFrame
# column_names = ['filepath', 'label']
# column_names += ['ProductType_' + name for name in le_prdtype.classes_]
# column_names += ['Weight_' + name for name in le_weight.classes_]
# column_names += ['HalalStatus_' + name for name in le_halal.classes_]
# column_names += ['HealthStatus_' + name for name in le_healthy.classes_]


# # Create the DataFrame
# main_imgs_results_small_model = pd.DataFrame(main_imgs_results_small_model, columns=column_names)
# main_imgs_results_small_model.head()

In [28]:
# # Check if any name from 'extracted_names' is not in 'df' and add it as a new column
# new_prdtype = list(set(all_imgs_results_big_model.columns) - set(main_imgs_results_small_model.columns))

# if len(new_prdtype)>0:
#     for col in new_prdtype:
#         main_imgs_results_small_model[col] = np.random.normal(loc=CONFIGS["MEAN_PRIOR"], scale=np.sqrt(0.1), size=main_imgs_results_small_model.shape[0])  # Initialize new columns

# main_imgs_results_small_model.head()  # Display the updated DataFrame for verificatio

In [29]:
# main_imgs_results_small_model['img_type'] = "existing"
# new_imgs_results_small_model['img_type'] = "new"
# all_imgs_results_small_model = pd.concat([main_imgs_results_small_model, new_imgs_results_small_model], axis=0)
# all_imgs_results_small_model.reset_index(drop=True, inplace=True)
# all_imgs_results_small_model.head()

In [30]:
# all_imgs_results_small_model.to_csv("all_imgs_results_small_model.csv", index=True)

# Scoring using updated Bayes model

In [31]:
# test images of the same product and different angles
test_img_paths = ["IMG_20230428_123708_jpg.rf.141ecd0cefaea75c0b7a5f281475dd6b.jpg",
                 "IMG_20230428_123659_jpg.rf.5e1b6c4caabe48cf36003cb4184ff380.jpg",
                 "IMG_20230428_123704_jpg.rf.5fc2415d06061ea102ef125a37bbc88c.jpg",
                 "IMG_20230428_123703_jpg.rf.6a9c54175f59238cdc83999cdee6dad4.jpg"]

In [32]:
from IPython.display import HTML, display

# Create HTML for images
html_images = ''.join([f'<img src="../../rshiny/www/all_images/{path}" width="150" />' for path in test_img_paths])

# Display images
display(HTML(html_images))

## Prepare average logits on product type prediction from big model

In [33]:
all_imgs_results_big_model.shape

(6728, 87)

In [34]:
big_model_input = all_imgs_results_big_model[all_imgs_results_big_model.filepath.isin(test_img_paths)]
big_model_input.reset_index(inplace=True, drop=True)
big_model_input.head()

,filepath,label,ProductType_BabyMilkPowder,ProductType_Babyfood,ProductType_BeehoonVermicelli,ProductType_BiscuitsCrackersCookies,ProductType_BreakfastCereals,ProductType_CannedBakedBeans,ProductType_CannedBeefOtherMeats,ProductType_CannedBraisedPeanuts,...,Weight_500-599g,Weight_600-699g,Weight_700-799g,Weight_800-899g,Weight_900-999g,HalalStatus_Halal,HalalStatus_NonHalal,HealthStatus_Healthy,HealthStatus_NonHealthy,img_type
0,IMG_20230428_123704_jpg.rf.5fc2415d06061ea102e...,Sugar_400-499g_NonHalal_NonHealthy,-2.494402,-1.791569,-2.931083,-2.731453,-3.042774,-2.230376,-2.247128,-2.003090,...,-0.627885,-1.355924,-1.050916,-1.172089,-0.900328,-0.272685,-0.707462,-1.430414,0.242681,new
1,IMG_20230428_123708_jpg.rf.141ecd0cefaea75c0b7...,Sugar_400-499g_NonHalal_NonHealthy,-1.968333,-1.419088,-0.547048,0.454092,-1.920190,-2.112387,-1.945395,-1.591247,...,-1.188427,-0.467083,-0.478968,-0.966611,-0.958989,-0.433235,0.064759,-1.440724,0.382754,new
2,IMG_20230428_123703_jpg.rf.6a9c54175f59238cdc8...,Sugar_400-499g_NonHalal_NonHealthy,-2.725633,-1.921768,-2.942325,-3.087705,-3.261580,-2.288211,-2.268609,-2.066198,...,-0.036972,-1.048772,-1.070416,-1.071048,-0.854639,-0.155133,-0.742337,-1.351143,0.248921,new
3,IMG_20230428_123659_jpg.rf.5e1b6c4caabe48cf360...,Sugar_400-499g_NonHalal_NonHealthy,-2.819379,-2.454517,-2.948536,-3.560223,-3.156401,-2.434649,-1.878743,-2.135395,...,-0.422941,-1.056378,-0.528758,-0.985187,-1.014767,-0.277433,-0.554033,-1.534313,0.263355,new


In [35]:
big_model_input.shape

(4, 87)

In [36]:
def get_max_col_name(row):
    return row.idxmax()

# individual predictions
big_model_input.filter(like="ProductType").apply(get_max_col_name, axis=1)

0     ProductType_SweetsChocolatesOthers
1    ProductType_BiscuitsCrackersCookies
2     ProductType_SweetsChocolatesOthers
3                      ProductType_Sugar
dtype: object

In [37]:
# predictions using average logits
big_model_input.filter(like="ProductType").mean().idxmax()

'ProductType_SweetsChocolatesOthers'

## Prepare predictions on product type and image quality from GPT

In [38]:
# choose the first image to call gpt
import base64
import requests
import sys
import pandas as pd
from openai import OpenAI

def encode_image(image_path):
    """Encodes an image to Base64."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

with open('/Users/liupeng/Desktop/Research/api.txt', 'r') as file:
    api_key = file.read()

image_path = os.path.join("../../rshiny/www/all_images/", test_img_paths[0])
base64_image = encode_image(image_path)

headers = {
      "Content-Type": "application/json",
      "Authorization": f"Bearer {api_key}"
    }

payload = {
          "model": "gpt-4-vision-preview",
          "messages": [
            {
              "role": "user",
              "content": [
                {
                  "type": "text",
                  "text": """ 
                  For this image, can you make a prediction for the following four labels? 
                  The product type is based on the appearance of the product, 
                  the weight is to recognize how heavy the product is by identify the weight information on the appearance,
                  the halal status is to recognize if the product is halal food or not, 
                  and the healthy status is to recognize if the product is healthy if it contains a red triangle shape based on Singapore standard.
                  Note that you can only choose one from the given options in the bracket for each label, even if you are not sure. 
                  also there is no need to add an extra note to your answer.
                    product type (
                        Babyfood
                        BabyMilkPowder
                        BeehoonVermicelli
                        BiscuitsCrackersCookies
                        Book
                        BreakfastCereals
                        CannedBakedBeans
                        CannedBeefOtherMeats
                        CannedBraisedPeanuts
                        CannedChicken
                        CannedFruits
                        CannedMushrooms
                        CannedPacketCreamersSweet
                        CannedPickles
                        CannedPorkLunchronMeat
                        CannedSardinesMackerel
                        CannedSoup
                        CannedTunaDace
                        CannedVegetarianFood
                        ChocolateMaltPowder
                        ChocolateSpread
                        CoffeePowder
                        CoffeeTeaDrink
                        CookingCreamMilk
                        CookingPastePowder
                        CornChip
                        DarkSoySauce
                        DriedBeans
                        DriedFruits
                        DriedMeatSeafood
                        DriedVegetables
                        FlavoredMilkDrink
                        Flour
                        FruitJuiceDrink
                        HerbsSpices
                        InstantMeals
                        InstantNoodlesMultipack
                        InstantNoodlesSingle
                        Jam
                        Kaya
                        KetchupChilliSauce
                        LightSoySauce
                        MaternalMilkPowder
                        MilkDrink
                        MilkPowder
                        Nuts
                        Oil
                        OtherBakingNeeds
                        OtherCannedBeansPeasNuts
                        OtherCannedSeafood
                        OtherCannedVegetables
                        OtherDriedFood
                        OtherHotBeveragesPowder
                        OtherNoodles
                        OtherSauceDressing
                        OtherSpreads
                        Pasta
                        PastaSauce
                        PeanutButter
                        Potatochips
                        PotatoSticks
                        RiceBrownOthers
                        RiceWhite
                        RolledOatsInstantOatmeal
                        Salt
                        SoftDrinksOtherReadyToDrink
                        SoupStock
                        Sugar
                        SweetsChocolatesOthers
                        TeaPowderLeaves
                        WetWiper
                    ),
                    weight ('400-499g', '700-799g', '500-599g', '200-299g', '100-199g',
                       '1-99g', '300-399g', '600-699g', '800-899g', '1000-1999g',
                       '900-999g', '3000-3999g'
                    ),
                    halal status ('NonHalal', 'Halal'),
                    healthy status ('NonHealthy', 'Healthy'),
                    
                    Also, provide the following assessment
                    image reflection (High, Medium, Low)
                    image clarity (High, Medium, Low)
                    prediction confidence for the product type (High, Medium, Low)
                    prediction confidence for the weight (High, Medium, Low)
                    prediction confidence for the halal status (High, Medium, Low)
                    prediction confidence for the healthy status (High, Medium, Low)

                    format your answer in json format so that it could be easily converted to dataframe
                    """
                },
                {
                  "type": "image_url",
                  "image_url": {
                    "url": f"data:image/jpeg;base64,{base64_image}"
                  }
                }
              ]
            }
          ],
          "max_tokens": 300
        }


response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
content = response.json()['choices'][0]['message']['content']

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

# Extracting the message content (the table with predictions)
content = response.json()['choices'][0]['message']['content']

# Parsing the content to extract prediction values
# Splitting the content string into lines and then parsing each line
lines = content.strip().split('\n')[2:]  # Skipping the header

# Parsing the strings to extract the relevant information
data_parsed = [line.replace('"', '').strip() for line in lines if ':' in line]
data_dict = {item.split(":")[0].strip(): item.split(":")[1].strip().strip(',') for item in data_parsed}

# Converting the dictionary into a DataFrame
df_from_strings = pd.DataFrame([data_dict])


In [39]:
df_from_strings

,product type,weight,halal status,healthy status,image reflection,image clarity,prediction confidence for the product type,prediction confidence for the weight,prediction confidence for the halal status,prediction confidence for the healthy status
0,Sugar,300-399g,Halal,NonHealthy,Low,High,High,High,Medium,Medium


In [40]:
col_names = ['product_type', 'weight', 'halal', 'health', 'image_reflection', 
                  'image_clarity', 'product_type_confidence', 'weight_confidence',
                  'halal_confidence', 'health_confidence']
parsed_data = df_from_strings
# Normalizing column names for consistency across all DataFrames
normalized_dfs = []
tmp_valid_paths = []


df = df_from_strings
if df.shape[1] != 0:
    if df.shape[1] != 10:
        print("--\n")
        print(i)
        print(df.shape[1])
        if df.columns[-1]=="Note":
            df.drop(columns=['Note'], inplace=True)
            df.columns = col_names
            normalized_dfs.append(df)
            tmp_valid_paths.append(tmp_paths[i])
        elif df.shape[1] == 11:
            df = df.iloc[:,:-1]
            df.columns = col_names
    else:
        # Renaming columns to have consistent names across all DataFrames
        df.columns = col_names

# Combining all DataFrames into a single DataFrame
gpt_pred_df = df

cols = gpt_pred_df.columns.tolist()
gpt_pred_df

,product_type,weight,halal,health,image_reflection,image_clarity,product_type_confidence,weight_confidence,halal_confidence,health_confidence
0,Sugar,300-399g,Halal,NonHealthy,Low,High,High,High,Medium,Medium


In [41]:
gpt_pred_label = gpt_pred_df['product_type'] + "_" + gpt_pred_df["weight"] + "_" + gpt_pred_df["halal"] + "_" + gpt_pred_df["health"]
gpt_pred_label.values[0]

'Sugar_300-399g_Halal_NonHealthy'

In [42]:
gpt_pred_label

0    Sugar_300-399g_Halal_NonHealthy
dtype: object

In [43]:
gpt_pred_confidence = "Product_" + gpt_pred_df['product_type_confidence'] + "-Weight_" + gpt_pred_df['weight_confidence'] + "-Halal_" + gpt_pred_df['halal_confidence'] + "-Health_" + gpt_pred_df['health_confidence']
gpt_pred_confidence.values[0]

'Product_High-Weight_High-Halal_Medium-Health_Medium'

## Call R script to perform scoring

In [44]:
import subprocess
import json
import os
import pandas as pd

# Set the R_HOME environment variable to the R home directory used by RStudio
os.environ['R_HOME'] = '/Library/Frameworks/R.framework/Resources'
os.environ['PATH'] = '/Library/Frameworks/R.framework/Resources/bin' + os.pathsep + os.environ['PATH']


In [45]:
# Your DataFrame
# df = pd.DataFrame({
#     'A': [1, 2, 3],
#     'B': [4, 5, 6]
# })

# Serialize DataFrame to JSON
# df_json = df.to_json()

# Pass the JSON directly as an argument (ensure it's not too large)
# input1 = df_json
input1 = big_model_input.to_json(orient='records')
input2 = gpt_pred_df.to_json(orient='records')

# The command (ensure correct paths)
command = ['Rscript', 'bayes_model_real_time.R', input1, input2]

# Run the command and capture the output
result = subprocess.run(command, capture_output=True, text=True)

# Check if the command was executed successfully
if result.returncode == 0:
    # Print stdout for debug messages and output
    # print("STDOUT from R:\n", result.stdout)
    # Parse the JSON output from the R script if needed
    try:
        output = json.loads(result.stdout)
        print("Output from R:", output)
    except json.JSONDecodeError:
        print("Failed to parse JSON output.")
else:
    # Print stderr for errors
    print("Error running R script:\n", result.stderr)


Output from R: {'pred': ['Sugar']}


In [46]:
output['pred'][0]

'Sugar'